# เกริ่นนำ

สวัสดีครับเพื่อนๆ Notebook นี้เป็นอีกหนึ่ง Workshop ของ www.thaikeras.com ที่มีจุดประสงค์เพื่อให้เรียนรู้วิธีการเขียนโปรแกรม Text Classification เบื้องต้นด้วย Keras นะครับ โดยนำโจทย์มาจาก
โดยโจทย์ของ Dataset Quora Insincere Classification ที่ต้องการคัดกรองกระทู้ที่มีเจตนาไม่ดี (ว่าง่ายๆก็คือ กระทู้ที่ทำให้คนทะเลาะกัน)  เพื่อความรวบรัดใน Notebook นี้จะขอเรียกกระทู้ที่มีเจตนาไม่ดีเหล่านี้ ย่อๆ ว่า **กระทู้ไม่ดี** ครับ

โดยผมมีกระทู้ที่อธิบายถึงปัญหานี้อย่างละเอียดไว้ และเพื่อนๆ สามารถดูเพิ่มเติม พูดคุยเกี่ยวกับ Notebook ได้ที่ [community ของเราครับ](https://thaikeras.com/community/%E0%B9%80%E0%B8%81%E0%B8%B5%E0%B9%88%E0%B8%A2%E0%B8%A7%E0%B8%81%E0%B8%B1%E0%B8%9A-kaggle/)

ถ้าไม่เข้าใจส่วนไหน ถามในเว็บบอร์ด หรือข้างล่าง Notebook นี้ พวกเรายินดีช่วยเหลือให้คำแนะนำทุกประเด็นครับผม

อนึ่ง Notebook นี้เป็นฉบับปรับปรุงและเพิ่มเติมจาก Notebook ที่มีชื่อว่า https://www.kaggle.com/sudalairajkumar/a-look-at-different-embeddings
ซึ่งเขียนโดยคุณ SRK ที่เป็น Kernel Grandmaster ดังนั้นเพื่อนๆ จะได้เรียนรู้จากอาจารย์ที่ดีที่สุดแน่นอนครับ ขอบคุณ คุณ SRK ไว้ ณ โอกาสนี้ด้วย แม้ Notebook นี้จะไม่ได้เป็น Notebook ที่ทำคะแนนได้สูง
ใน Leaderboard ทว่าเท่าที่คุยกับเพื่อนๆ ที่ร่วมแข่งมาด้วยกัน ผู้เข้าแข่งขันส่วนใหญ่ต่างก็ใช้ Notebook นี้เป็นจุดเริ่มต้นและปรับปรุงต่อจนกระทั่งได้รับเหรียญรางวัลครับ

Notebook จะใช้เวลาทำงานทั้งหมดตั้งแต่ต้นจนจบราวๆ 20นาทีกว่าๆ ครับ

## สิ่งที่จะได้เรียนจาก Workshop นี้

- เรียนรู้การเขียนโปรแกรม Text Classification ด้วย Keras โดยใช้ Sequential Model พื้นฐานเช่น LSTM / GRU และการเตรียมข้อมูลเบื้องต้นด้วย Keras Tokenizer

- ได้เรียนรู้การใช้ Word Embedding ทั้งหลายซึ่งเป็นหัวใจของงานด้าน NLP ไม่ว่าจะเป็น GloVe หรือ FastText เบื้องต้น

- เรียนรู้การวัดผลของ Model บนปัญหา **คลาสไม่สมดุล (imbalance class)** ซึ่งอาจนำไปสู่ความเข้าใจที่ผิด ถ้าวัดผลด้วย Accuracy metric หรือ "ค่าความถูกต้อง" ที่นิยมใช้ทั่วไป

- เรียนรู้วิธีที่เรียกว่า **Ensemble** อย่างง่าย ที่นำ Models หลายๆ ตัว มาช่วยกัน Classify ทำให้ความแม่นยำเพิ่มขึ้นมากกว่าการใช้ Model ใด Model หนึ่งเพียงตัวเดียว

### Workshops อื่นๆ โดยทีมงาน

- [ทำ Project Data Science เบื้องต้นจากการวิเคราะห์และทำนายผู้รอดชีวิตจากเรือไททานิก](https://www.kaggle.com/pednoi/data-science)

- [ทดลองเขียน Python และ Numpy](https://www.kaggle.com/ratthachat/python-numpy-deep-learning)

## เริ่มต้น code กันเลยครับ

In [ ]:
'''เริ่มต้นจากการ import libraries ทั้งหลายที่จำเป็นต้องใช้งานเข้ามานะครับ'''
'''บาง libraries พวกเราอาจจะยังไม่เคยใช้ แต่ถ้าเห็นวิธีการใช้บ่อยๆ ก็จะคุ้นชินไปเองครับ'''
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

np.set_printoptions(suppress=True)

'''Keras libraries เหล่านี้เป็นมาตรฐานที่เราแทบจะ Copy ไปใช้กับทุกปัญหาได้เลยครับ'''
# Tokenizer ใช้จัดการตัดสตริงให้เป็นคำ และ map แต่ละคำเป็นตัวเลข ท้ายที่สุดสตริงจะเปลี่ยนเป็น Sequence of numbers
from tensorflow.keras.preprocessing.text import Tokenizer
# Pad_sequences ใช้สำหรับเติม 0 ลงไปให้ทุกสตริงสุดท้ายมีความยาวเท่ากัน
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

## ทำความรู้จักและจัดการข้อมูลเบื้องต้น

In [ ]:
# โหลดข้อมูล train/test โดยใช้ panda.read_csv ฟังก์ชันที่ใช้บ่อยที่สุดใน Data Science :)
train_df = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test_df = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

เรามีข้อมุล train ราว 1.3 ล้านกระทู้ และต้องทดสอบบนกระทู้ test ราวๆ 3.76 แสนกระทู้ ก็ถือว่ามีข้อมูลที่เยอะพอควรนะครับ

หลังจากโหลดข้อมูล train/test data ขึ้นมาแล้ว เราลองดูเบื้องต้นว่า train/test ถูกจัดเรียงอย่างไรนะครับ ซึ่งจะเห็นได้ว่าข้อมูล train แต่ละตัวมี field สำคัญอยู่สอง columns คือ "question_text" หรือหัวข้อกระทู้ และ "target" ที่บ่งบอกว่ากระทู้นี้มีเจตนาที่ดีหรือไม่ โดย target = 1 แปลว่ามีเจตนาไม่ดีครับ


In [ ]:
train_df.head(10) # ลองดูซิ ว่าข้อมูลสอนมีหน้าตาเป็นอย่างไร

ข้อมูลจะประกอบไปด้วย column รหัสเฉพาะของแต่ละคำถาม qid และประโยคคำถามของแต่ละกระทู้ question_textและค่า target ที่ระบุว่าเป็นกระทู้ที่ดีหรือไม่ 

โดยเราจะใช้ข้อมูลในส่วนของ question_text และ target เท่านั้นโดยไม่มีความจำเป็นต้องใช้ qid ครับ

In [ ]:
test_df.head(5) # ข้อมูลทดสอบหรือ test data จะเหมือน train ยกเว้นเราจะไม่รู้ target ที่เราต้องการทำนาย

เนื่องจากข้อมูลที่ Model เราต้องทำความเข้าใจนั้นอยู่ใน field **question_text** ซึ่งประกอบไปด้วย strings เป็นภาษามนุษย์ เราจึงต้อง preprocess เบื้องต้นเพื่อให้ Model หรือ computer ของเราสามารถทำความเข้าใจมันได้ครับ

ในที่นี่ เราจะ preprocess ข้อมูลอย่างง่าย ตาม steps ต่อไปนี้

1) ก่อนอื่นแบ่ง train data ออกเป็นสองส่วนคือ train data (ชุดใหม่) และ validation data หรือบางครั้งเรียกว่า development data โดย validation data นั้นมีจุดประสงค์เพื่อเอาไว้ใช้ทดสอบประสิทธิภาพของโมเดลเราว่าดีหรือไม่ดีเบื้องต้น (ก่อนที่จะนำไปทดสอบจริงบน test set)

2) ถ้ากระทู้ไหนไม่มีข้อมูลใน field **question_text** เราก็เพิ่งประโยค  '_na_' ลงไป เพื่อให้การเขียนโปรแกรมในขั้นถัดๆ ไปทำได้ง่าย ไม่ต้องมาจัดการ missing values

3) ใช้ Tokenizer : ตัดประโยคให้เป็นคำๆ ทำ mapping ศัพท์แต่ละคำให้เป็นตัวเลข ซึ่งทำให้ท้ายที่สุด เราจะเปลี่ยนข้อความในทุกๆ กระทู้ให้เป็น Sequence of numbers ซึ่่ง computer สามารถนำไปทำความเข้าใจได้โดยง่าย
 * ทั้งนี้เราต้องกำหนดด้วยว่าจะให้ model สามารถจดจำศัพท์ได้กี่คำ (ในโปรแกรมข้างล่างเราเรียกตัวแปรนี้ว่า **max_features**) บางครั้งการจำศัพท์ที่ใช้บ่อยมากเกินไป หรือใช้แค่ 1-2 ครั้งบนตัวอย่างทั้งหมดที่เรามีอาจไม่มีประโยชน์ในการทำความเข้าใจว่า ประโยคหนึ่งๆ นั้นมีเจตนาดีหรือไม่

4) เพื่อความง่ายเราจะกำหนดให้ทุกๆ sequence of numbers ดังกล่าวมีความยาวเท่าๆ กันทั้งหมด (เรียกค่าความยาวนี้ว่า **maxlen**) ดังนั้นกระทู้ไหนที่ยาวเกินไป เราจะตัดประโยคส่วนหัวทิ้ง (จะเลือกตัดส่วนท้ายก็ได้) และถ้าประโยคไหนสั้นเกินไป เราจะเติมตัวเลข 0 ลงไปให้สุดท้ายยาวเท่ากับ maxlen


In [ ]:
%%time
## 1) ใช้ฟังก์ชัน train_test_split เพื่อแบ่งข้อมูลออกเป็น train/valid data
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)

## กำหนดค่าคงที่ต่างๆ ซึ่งสามารถปรับจูนได้เพื่อประสิทธิภาพที่ดีขึ้น ในภายหลัง
embed_size = 300 # ชนาด dimension ของ word vectors
max_features = 100000 # จำนวนคำศัพท์ที่เราจะให้ model รู้จัก
maxlen = 70 # กำหนดให้ความยาวของทุกประโยคเท่ากันที่ 70

## 2) ดึงเฉพาะข้อความกระทู้ขึ้นมา รวมทั้งจัดการ missing values อย่างง่าย
train_X = train_df["question_text"].fillna("_na_").values
val_X = val_df["question_text"].fillna("_na_").values
test_X = test_df["question_text"].fillna("_na_").values

## 3) ใช้ Tokenizer
#3.1) สร้าง object tokenizer ขึ้นมา ระบุคำศัพท์สูงสุดที่ต้องการจำ
tokenizer = Tokenizer(num_words=max_features)
#3.2) ให้ tokenizer เรียนรู้ศัพท์ใน training data
tokenizer.fit_on_texts(list(train_X))
#3.3) ใช้ tokenizer เปลี่ยนจาก string เป็น sequence of numbers
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## 4) Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## 5) เก็บค่า targets หรือ labels ไว้ในตัวแปร y
## เรามองเกือบทุกปัญหาใน machine learning เป็นการหาฟังก์ชันที่ดีในการ map จาก X --> y
train_y = train_df['target'].values
val_y = val_df['target'].values

In [ ]:
'''ตอนนี้ข้อมูลเราจะอยู่ในรูป Sequence of numbers ที่ีมีความยาว maxlen = 70 แล้วครับ
สังเกตว่า เราแบ่งข้อมูล 10% ไปเป็น Validation data, ทำให้ training data เหลืออยู่ 1.175 ล้านกระทู้ครับ
'''
print('ขนาดข้อมูลหลังแปลง : ',train_X.shape, val_X.shape, train_y.shape, val_y.shape, '\n')
'''เราลอง print train_X[0] เพื่อดูตัวอย่าง sequence of numbers, สังเกตว่าเราแปะ 0 ไว้ที่ข้างหน้าเพื่อให้ความยาวครบ 70 โดยแท้จริงแล้วเราเลือกแปะ 0 ไว้ข้างหลังก็ได้เช่นกัน '''
print('ตัวอย่างข้อมูลสอนตัวแรกจะอยู่ในรูป sequence of numbers ความยาว maxlen = %d ดังนี้ : ' % (maxlen))
print(train_X[0])

In [ ]:
'''ปัจจัยหนึ่งที่ทำให้ปัญหาคัดกรองกระทู้ ทำได้ยากก็คือ สัดส่วนของกระทู้ที่เจตนาไม่ดีนั้น มีน้อยกว่ากระทู้ปกติมาก 
(ซึ่งทำให้โมเดลของเราทำความเข้าใจรูปแบบกระทู้ที่ไม่ดีได้ลำบาก เพราะตัวอย่างส่วนใหญ่จะเป็นกระทู้ดี)

สัดส่วนของกระทู้ที่ไม่ดี มีเพียงราว 6.2% เท่านั้น
'''
print('สัดส่วนกระทู้ไม่ดีเท่ากับ %.4f ' % (sum(train_y)/len(train_y)))

# Word Embedding
Word embedding หรือการเปลี่ยนคำศัพท์ภาษามนุษย์ให้อยู่ในรูปเวกเตอร์ทางคณิตศาสตร์ ซึ่งทำให้คอมพิวเตอร์เข้าใจความสัมพันธ์ของศัพท์แต่ละคำได้อย่างเป็นระบบ เราสามารถเรียน Word Embedding ได้จากข้อมูลเช่นเดียวกับ parameters อื่นๆ ในโมเดลหรือจะใช้ word vectors ที่ถูกเตรียมมาให้แล้ว (pretrained) ก็ได้เช่นกัน 

โดยทีมงานจาก Google, Facebook และ Stanford ต่างก็ได้เตรียม (pre-trained) word embedding vectors ไว้ให้เรียบร้อยแล้ว ในชื่อ Word2Vec, FastText, และ GloVe ตามลำดับ และทาง Kaggle เองก็ได้เตรียม pretrained vectors ไว้ให้ใช้งานได้ทันทีใน Notebook นี้เรียบร้อยแล้วครับ

สามารถอ่านรายละเอียดเพิ่มเติมได้ในกระทู้นี้ครับ

https://thaikeras.com/community/%E0%B9%80%E0%B8%81%E0%B8%B5%E0%B9%88%E0%B8%A2%E0%B8%A7%E0%B8%81%E0%B8%B1%E0%B8%9A-kaggle/quora-insincere-questions-classification-detect-toxic-content-to-improve-online-conversations/


Deep Learning ขึ้นมาเพื่อเรียนรู้จากข้อมูลกระทู้ต่างๆ และจะยังไม่ใช้ pretrained word vectors ครับ

## เริ่มต้นสร้าง model ด้วย Keras

Keras เป็นการเขียนโปรแกรมสร้าง Deep Learning model ที่ง่ายดายและมีประสิทธิภาพสูง และเราสามารถนำ pretrained word embedding vectors มาใช้ได้อย่างสะดวก

### ทำความเข้าใจการวัดผล และ baseline เมื่อเราไม่ใช้ word embedding
**ก่อนอื่นเราจะทำความเข้าใจเรื่องการวัดผลในปัญหานี้ และลองดูประสิทธิภาพแบบไม่ใช้ pretrained word embedding กันก่อนครับ **

ในตัวอย่างแรกเราจะลองสร้าง model Bidirectional GRU ซึ่งเป็น Sequential Model มาตรฐานสำหรับ Deep Learning  กรณีที่เราไม่ใช้ pretrained word embedding เราก็จะ train word embedding layer ขึ้นมาเองจากข้อมูล training data ที่เรามีครับ 

In [ ]:
'''โค้ดต่อไปนี้ เป็นโค้ดของ Keras ที่สามารถใช้สร้าง Deep Sequential Neural Network Model ขึ้นมาได้อย่างง่ายดาย 
โดย Deep Neural Networks นั้นจะประกอบไปด้วย Neural Layers ต่างๆ มาประมวลผลต่อเนื่องกัน โดยเพื่อนๆ สามารถศึกษา
รายละเอียดของ Layers ต่างๆ ได้จาก Course online เช่น DeepLearning.ai บน coursera.org หรือ course ของ ThAIKeras ในอนาคต

ลักษณะการสร้าง Layers ของ Keras จะมีรูปแบบตายตัวดังนี้

output_tensor = NewLayer(layer_parameters)(input_tensor)

โดยเราสามารถนำ Layers ต่างๆ อาทิเช่น Embedding, GRU, MaxPool, หรือ Dense มาต่อกันได้อย่างอิสระ คล้ายกับการต่อ Lego ดังโค้ดข้างล่าง
'''

## นิยามโมเดลด้วยการนำ Layers มาเชื่อมกัน
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size)(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)

## สั่งสร้าง model ด้วยการกำหนด input และ output tensors
model = Model(inputs=inp, outputs=x)

# compile คือการระบุวิธีการฝึกโมเดล ว่าต้องการ loss/metric ประเภทไหน และใช้ optimizer แบบไหน
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# print โมเดลสรุปออกมา
print(model.summary())

จาก model summary ข้างบนสังเกตว่าเรามี parameters ของ neural networks ที่ต้องเรียนรู้ถึง 30 กว่าล้านตัวแปรเลยทีเดียว โดยเกือบทั้งหมดคือตัวแปรของ embedding layers จำนวน 30ล้านตัวแปร ทั้งนี้เพราะเราระบุจำนวนคำศัพท์เท่ากับ 1แสนคำ และคูณกับ 300 ซึ่งเป็น dimension ของ vector 

เมื่อเราสร้างโมเดลเสร็จแล้ว เราสามารถสั่ง train หรือให้โมเดลเรียนรู้จากข้อมูล training data ของเราได้ง่ายๆ ทันทีเลยครับด้วยคำสั่ง `model.fit` โดยระบุ batch size เป็น 512 (จำนวน data ที่สุ่มมาทำ backpropagation ในแต่ละครั้ง) และรันทั้งหมด 3 epochs (อ่านข้อมูล training data จำนวน 1.17 ล้านกระทู้ทั้งหมด 3 รอบ)

In [ ]:
## Train the model 
model.fit(train_X, train_y, batch_size=512, epochs=3, validation_data=(val_X, val_y))

หลังจากเทรนเสร็จแล้ว เราใช้คำสั่ง `model.predict` เพื่อทำนายข้อมูล validation data เพื่อที่จะได้ตรวจสอบว่าเราทำนายได้แม่นยำมากน้อยแค่ไหน 

In [ ]:
pred_noemb_val_y = model.predict([val_X], batch_size=1024, verbose=1)[:,0]

ก่อนอื่นเราลอง print ค่าที่ได้จาก `model.predict` ออกมาดูว่ามีหน้าตาเป็นแบบไหนก่อนครับ

In [ ]:
print(pred_noemb_val_y.shape)
print('ตัวอย่างคำทำนาย 10 กระทู้แรกใน validation data: ' ) # print ดูที่เราทำนาย 10 cases แรก
print(pred_noemb_val_y[:10])

สังเกตว่าคำทำนายมีทั้งหมด 130,613 ค่า ซึ่งก็ตรงกับจำนวน validation data ที่เราต้องการทดสอบ 

นอกจากนี้ค่าที่ได้จะอยู่ในช่วง [0,1] นี่เป็นเพราะ layer สุดท้ายใน model ของเราเป็น layer sigmoid และเราตีความหมายได้ว่าเป็น "ความน่าจะเป็น" ที่แต่กระทู้จะเป็นกระทู้ที่ไม่ดีครับ ซึ่งจากตัวอย่างคำทำนายใน 10 กระทู้แรกของ validation set เราพบว่ามีเพียง 1 กระทู้ที่น่าจะเป็น "กระทู้ไม่ดี" นั่นคือกระทู้ที่ 6 ซึ่งมีความน่าจะเป็นมากกว่า 50% ครับ

In [ ]:

print('ค่า target จริงของ 10 กระทู้แรก: ', val_y[:10]) # print ดู label จริง

 ## F1 : มาตรวัดที่ดีกว่า "ความถูกต้อง" (accuracy)

- ใน 10 กระทู้แรกนั้นเหมือนคำทำนายเราจะแม่นเลยทีเดียวครับ เพราะมีเพียงกระทู้ที่ 6 ที่เป็นกระทู้ไม่ดีจริงๆ ดั่งคำทำนาย 
- แม้จะดูเหมือนแม่น แต่จริงๆ แล้วต้องระวังครับ ในปัญหาที่จำนวนตัวอย่างแต่ละ**คลาสไม่สมดุลอย่างมาก (very imbalance class)** แบบนี้ เรามีแต่ตัวอย่างคลาส 0 หรือ "กระทู้ดี" เต็มไปหมด (94%) ดังนั้นการตอบว่า **กระทู้ดี** เป็นเรื่องง่ายมากครับ ดังนั้นจริงๆ แล้วเราต้อง Focus การตอบกระทู้ไม่ดี ให้ถูกต้องมากกว่า
- ในปัญหานี้ ถ้าโมเดลเราตอบว่าทุกกระทู้ "ดี" ทั้งหมด ก็จะได้ความ**ถูกต้อง**ถึงราวๆ 94% ซึ่งดูเผินๆ เหมือนดีมาก แต่ความจริงแล้วเราเรียนรู้อะไรไม่ได้เลย ดังนั้น **"ค่าความถูกต้อง" หรือ accuracy** จึงเป็นการวัดผลที่อาจไม่เหมาะสม และทำให้เราเข้าใจผิดคิดว่าโมเดลเราทำงานได้ดีแล้วครับ 
- การวัดผลที่เหมาะสมกว่ามีหลายวิธี ซึ่งวิธีที่ Quora เลือกใช้ในการประลองครั้งนี้ก็คือการวัดด้วย F1 Score ครับ (ดูรายละเอียดข้างล่าง หรือที่ https://en.wikipedia.org/wiki/F1_score) ในที่นี้เราเรียก f1_score function จาก scikit learn library ได้เลยครับ

### หมายเหตุ: ว่าด้วยเรื่องการวัดผล
ในบทความนี้จะใช้คำศัพท์ต่อไปนี้ในความหมายที่แตกต่างกัน เพื่อนๆ ระวังสับสนนะครับ: 
- ความถูกต้อง (accuracy)
- ความแม่นยำ (precision)
- ความครอบคลุม (recall)

สามคำนี้จะให้ความหมายของประสิทธิภาพของคำนายที่ไม่เหมือนกัน นอกจากนี้เราจะใช้ F1 Score ซึ่งเป็นคล้ายๆ ค่าเฉลี่ยของ precsion และ recall เป็นมาตรวัดหลัก

ก่อนจะไปเจาะลึกถึงเรื่องการวัดผลแบบ F1 เราลองดูประสิทธิภาพของคำทำนายทั้งหมด 1.3 แสนกระทู้ใน validation data ว่าแม่นยำแค่ไหน โดยในการทำนายเบื้องต้นนั้น เราจะทำนายกระทู้ที่มีความน่าจะเป็นมากกว่า 50% ว่า "ไม่ดี" ครับ

In [ ]:
from sklearn import metrics

thresh = 0.50
pred_noemb_val_y_int = pred_noemb_val_y > thresh
print('ค่า accuracy เท่ากับ %.4f' % (sum(pred_noemb_val_y_int == val_y)/len(val_y)))
print('ค่า F1 เท่ากับ %.4f' % (metrics.f1_score(val_y, (pred_noemb_val_y>thresh).astype(int)) ) )

### หมายเหตุ : การรัน Keras แต่ละครั้งอาจได้ผลต่างกันเล็กน้อย
เนื่องจาก keras model นั้นทุกครั้งที่รันจะได้ผลต่างกันนิดหน่อย ดังนั้นค่าที่ผมได้จากการทดลองระหว่างเขียนบทความนี้นั้น ก็จะต่างจากค่าที่เพื่อนๆ เห็นอยู่บ้าง

ดังนั้นตัวเลขที่ผมใช้อธิบายใน notebook นั้นจะขอใช้ตัวเลขคร่าวๆ แทนตัวเลขเป้ะๆ นะครับ เช่นใช้ 60%++ แทน 62.19% หรือ 63.61% เป็นต้น

## ทำความเข้าใจ F1 จาก Precision และ Recall
จากผลข้างบนค่าความแม่นยำเราเพิ่มจาก 94% (ทาย 0 หมด) เป็น 95%++ และค่า F1 เราเท่ากับ 60%++ จากตรงนี้ เข้าใจว่าเพื่อนๆ อาจสับสนเหมือนกันว่าจะแปลความหมายอย่างไรดี

เพื่อความง่ายเราอาจมองว่าค่า F1 เป็นค่าเฉลี่ยระหว่างสองค่าที่สำคัญในการทำนาย class1 หรือ กระทู้ไม่ดีดังนี้ครับ

- ค่า**ความแม่นยำ**ของคำทำนายเรา (precision) : จากที่เราทาย class 1 ไปนั้น "คำทำนาย" ของเราถูกกี่ตัว
- ค่า**ความครอบคลุม**ของคำทำนายเรา (recall) : กระทู้ที่ไม่ดี class 1 ทั้งหมดนั้นถูกทายถูกจำนวนกี่ตัว

สำหรับเพื่อนๆ ที่เพิ่งเคยเจอ precision และ recall เป็นครั้งแรกอาจสับสนว่ามันต่างกันตรงไหนนะครับ ที่แตกต่างกันคือ "ตัวหารครับ" โดย precision นั้นเทียบจาก "จำนวน class1 ที่เราทำนาย" ส่วน recall นั้นเทียบจาก "จำนวน class1 จริงๆ" (ความสามารถในการครอบคลุม class1 ของโมเดลเรา)  

สังเกตว่าถ้าเราทำนายแค่กระทู้เดียวที่เรามั่นใจมากๆ แล้วถูกเราก็จะมี precision 100% แต่ recall ก็จะต่ำมากเพราะเราครอบคลุมได้แค่ 1 กระทู้จากกระทู้ class1 ที่มีมากมายครับ ดังนั้นค่า F1 ที่คล้ายๆเป็นค่าเฉลี่ยของทั้งสองค่านี้ก็จะยังต่ำด้วย

ดังนั้นการที่เราพิจารณาตัวเลข accuracy, precision & recall นั้นก็จะทำให้เราเข้าใจโมเดลเราในสามมุมมองที่แตกต่างกัน

อนึ่ง ทำความเข้าใจ precision / recall / F1 ไว้ให้ดีๆ ครับเพราะเป็นมาตรวัดที่สำคัญมากๆ ใน AI และ Data Science ครับ และเพื่อนๆ จะได้เจอมันบ่อยๆ อีกแน่นอนในอนาคต

![รูป precision & recall](https://cdn-images-1.medium.com/max/1600/1*pOtBHai4jFd-ujaNXPilRg.png)

In [ ]:
count = np.logical_and(pred_noemb_val_y_int == val_y, val_y == 1) # นับจำนวนที่เราทำนาย class1 ถูกต้อง

print('ค่า precision : %4f '% (sum(count)/sum(pred_noemb_val_y_int == 1)))
print('ค่า recall : %4f '% (sum(count)/sum(val_y == 1)))

เพื่อนๆ แปลความหมายค่า precision / recall ที่ได้ ได้ไหมครับ? และค่า F1 เป็นค่ากลางๆ ระหว่างสองค่านี้จริงไหมครับ

ตอนนี้เพื่อนๆ ก็คงเข้าใจ F1 score มากขึ้นแล้วนะครับผม

## ตัวเลข 50% threshold เหมาะสมไหม?
ดูเผินๆ **การทำนายกระทู้ว่าไม่ดี** ถ้ามีความน่าจะเป็นว่าไม่ดีมากกว่า 50% ก็ดูสมเหตุสมผลดีแล้ว แต่อย่าลืมว่าปัญหาของเราเป็นปัญหา imbalance ซึ่งสัดส่วนกระทู้ที่ไม่ดีมีน้อยมาก model ของเราอาจลำเอียงให้ค่าความน่าจะเป็นน้อยโดยเฉลี่ย

เพื่อทำความเข้าใจเราลอง print ค่าความน่าจะเป็นของข้อมูลเฉพาะ class1 ใน validation data 10 ตัวอย่างแรกมาดูกันครับ

In [ ]:
idx_1 = np.where(val_y == 1)[0] # หา id เฉพาะกระทู้ไม่ดี (class1) ใน validation data
print(pred_noemb_val_y[idx_1[:10]]) # print คำนาย 10 cases แรกของกระทู้ไม่ดี
print(val_y[idx_1[:10]]) # 1 ทั้งหมด (กระทู้ไม่ดีทั้งหมด)

ตัวเลขเป็นอย่างไรบ้างครับ ตัวเลขที่เพื่อนๆ เห็นจะต่างจากตัวเลขที่ผมได้จากในขณะเขียนบทความนี้เล็กน้อย แต่รวมๆ แล้วน่าะจะมีกระทู้ class1 ที่ค่าความน่าจะเป็นต่ำกว่า 50% อยู่ 2-4 กระทู้ ซึ่งทำให้ค่า recall ใน 10 cases แรกของเราอยู่ที่ราวๆ 60%-80% ถ้าเราใช้ threshold = 50% ในการตัดสินว่ากระทู้นั้นเป็นกระทู้ไม่ดีหรือไม่

สังเกตว่าถ้าเราลด threshold ลงอีก เช่นให้ threshold = 30% หรือ 

> กระทู้ที่มีความน่าจะเป็น > 30% นั้น 'ไม่ดี'

เราก็จะได้ค่า recall มากขึ้นเป็น 80-90% เลยทีเดียวครับ แต่ค่า F1 ของเราอาจจะลดลงก็ได้เพราะค่า precision อาจจะต่ำลง

เอ แล้วเราจะรู้ได้อย่างไรล่ะ ว่าค่า threshold ไหนที่สร้างสมดุลระห่ว่าง precision และ recall ได้ดีที่สุด นั่นคือได้ F1 มากที่สุด??

วิธีหนึ่งที่เราทำได้และเป็นวิธีมาตรฐานที่ผู้เข้าแข่งขันเกือบทุกคนใช้ก็คือ เราจะวนลูปทดลองค่า threshold ไปเรื่อยๆ ดูว่า threshold ไหนให้คะแนน F1 Score เยอะที่สุดบน  validation data 
แล้วเราก็เลือก threshold นั้นดังแสดงในโค้ดข้างล่างนี้ครับ

In [ ]:
best_thresh = 0.2
max_f1 = 0
for thresh in np.arange(0.2, 0.601, 0.01):
    thresh = np.round(thresh, 2)
    current_f1 = metrics.f1_score(val_y, (pred_noemb_val_y>thresh).astype(int))
    print("ค่า F1 ที่ threshold %.2f is %.4f" % (thresh, current_f1) )
    
    if current_f1 > max_f1:
        best_thresh = thresh
        max_f1 = current_f1

print('ค่า threshold ที่ดีที่สุดคือ %.2f ให้ค่า F1 เท่ากับ %.4f' % (best_thresh,max_f1))

ค่า threshold ที่ดีทีสุดไม่ใช้ 50% และค่า F1 ที่ดีที่สุดนั้นมากกว่าค่า F1 ที่ threshold=50% อยู่พอสมควรเลยครับ!

### หมายเหตุ
วิธีการนี้อาจทำให้ค่า F1 ของเรา overfit กับ validation data ได้ อย่างไรก็ดีถ้า validation data ของเรามีจำนวนมาก ค่า F1 นี้ก็จะ overfit ไม่มากครับ (ในทางปฏิบัติผู้เข้าแข่งขันที่ทำคะแนนได้สูง จะมีวิธีหาค่า threshold ที่ซับซ้อนกว่านี้เพื่อให้ overfit น้อยที่สุด)

## ทำนาย test data
เมื่อเราทดสอบบน validation data และได้ค่า threshold ที่น่าพอใจแล้ว เราก็เอาความรุ้ที่ได้มาทำนาย class1 หรือ กระทู้ไม่ดี บน test data ทั้ง 3.75 แสนกระทู้ได้เลยครับ

In [ ]:
pred_noemb_test_y = model.predict([test_X], batch_size=1024, verbose=1)

เพื่อการประหยัดเนื่อที่ เราใช้คำสั่งต่อไปนี้ในการล้างข้อมูลหรือตัวแปรที่เราจะไม่ใช้แล้วครับ

In [ ]:
del model, inp, x
import gc; gc.collect()
time.sleep(10)

# ทดลองใช้ pretrained-embedding

หลังจากเราได้ประสิทธิภาพจาก GRU Baseline ที่ไม่ได้ใช้ pre-trained embedding แล้ว เราจะลองใช้ pretrained word embedding ใน keras models กันดูบ้างว่าจะทำให้ประสิทธิภาพดีขึ้นเท่าไร

ทั้งนี้ Kaggle & Quora ได้เตรียม pre-trained word embedding vectors ให้เราทั้้งหมด 4 ชุด


In [ ]:
!ls ../input/quoratextemb/

ซึ่งมีที่มาคร่าวๆ ดังนี้ครับ
 * GoogleNews-vectors-negative300 - https://code.google.com/archive/p/word2vec/
 เป็น embedding ของ Google ใช้ algorithm word2vec
 
 * glove.840B.300d - https://nlp.stanford.edu/projects/glove/
 เป็น embedding ของมหาวิทยาลัย Stanford ใช้ algorithm GloVe
 
 * paragram_300_sl999 - https://cogcomp.org/page/resource_view/106
 เป็น embedding ของมหาวิทยาลัย Pennsylvania ใช้ algorithm Paragram
 
 * wiki-news-300d-1M - https://fasttext.cc/docs/en/english-vectors.html
 เป็น embedding ของ Facebook ใช้ algorithm FastText
 
 ถ้าอยากดูรายละเอียดเพิ่มเติมดูได้ที่ [kernel](https://www.kaggle.com/sbongo/do-pretrained-embeddings-give-you-the-extra-edge) นี้ครับ

### Glove Embeddings
เราลองใช้ GloVe กันก่อนเลยครับ โดยขั้นตอนการสร้าง keras model แทบจะเหมือนเดิมทุกอย่าง ยกเว้นการโหลด weights ของ pretrained vectors เข้าไปเท่านั้น

In [ ]:
'''ฟังก์ชันในการโหลด pre-trained word embedding vectors

INPUTS: 
(1) word_index : dictionary ที่ map จาก "ศัพท์ภาษาอังกฤษ" เป็น "ตัวเลข" ได้จาก keras tokenizer
(2) max_words : จำนวนคำศัพท์ที่เราต้องการ ในที่นี่้มีค่าเท่ากับ max_features
(3) embed_size ขนาด dimension ของ vectors ซึ่งเรากำหนดค่ามาตรฐานสากล = 300

OUTPUT: embedding matrix ขนาด (max_words x embed_size)

'''

def load_glove_fast(word_index, max_words=max_features, embed_size=300):
    EMBEDDING_FILE = '../input/quoratextemb/glove.840B.300d/glove.840B.300d.txt'
    emb_mean, emb_std = -0.005838499, 0.48782197

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_words, embed_size))
    with open(EMBEDDING_FILE, 'r', encoding="utf8") as f:
        for line in f:
            word, vec = line.split(' ', 1)
            if word not in word_index:
                continue
            i = word_index[word]
            if i >= max_words:
                continue
            embedding_vector = np.asarray(vec.split(' '), dtype='float32')[:300]
            if len(embedding_vector) == 300:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

def load_fasttext_fast(word_index, max_words=max_features, embed_size=300):
    EMBEDDING_FILE = '../input/quoratextemb/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    emb_mean, emb_std = -0.0033470048, 0.109855264

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_words, embed_size))
    with open(EMBEDDING_FILE, 'r', encoding="utf8") as f:       
        for line in f:
            if len(line) <= 100:
                continue
            word, vec = line.split(' ', 1)
            if word not in word_index:
                continue
            i = word_index[word]
            if i >= max_words:
                continue
            embedding_vector = np.asarray(vec.split(' '), dtype='float32')[:300]
            if len(embedding_vector) == 300:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

def load_para_fast(word_index, max_words=max_features, embed_size=300):
    EMBEDDING_FILE = '../input/quoratextemb/paragram_300_sl999/paragram_300_sl999.txt'
    emb_mean, emb_std = -0.0053247833, 0.49346462

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_words, embed_size))
    with open(EMBEDDING_FILE, 'r', encoding="utf8", errors="ignore") as f:        
        for line in f:
            if len(line) <= 100:
                continue
            word, vec = line.split(' ', 1)
            if word not in word_index:
                continue
            i = word_index[word]
            if i >= max_words:
                continue
            embedding_vector = np.asarray(vec.split(' '), dtype='float32')[:300]
            if len(embedding_vector) == 300:
                embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [ ]:
%%time
'''โหลด weights ของ pre-trained vectors มาเก็บไว้ในตัวแปร'''

embedding_matrix =  load_glove_fast(tokenizer.word_index)


In [ ]:
'''สร้าง Deep Learning Model ด้วย Keras'''
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp) # ต่างจากเดิมที่บรรทัดนี้บรรทัดเดียวครับ
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=3, validation_data=(val_X, val_y))

In [ ]:
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)


In [ ]:
best_thresh = 0.2
max_f1 = 0
for thresh in np.arange(0.2, 0.601, 0.01):
    thresh = np.round(thresh, 2)
    current_f1 = metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))
    print("ค่า F1 ที่ threshold %.2f is %.4f" % (thresh, current_f1) )
    
    if current_f1 > max_f1:
        best_thresh = thresh
        max_f1 = current_f1

print('ค่า threshold ที่ดีที่สุดคือ %.2f ให้ค่า F1 เท่ากับ %.4f' % (best_thresh,max_f1))

จะเห็นว่าผลลัพธ์ที่ได้จะดีกว่าเวอร์ชั่นก่อน ที่ไม่ใช้ pretrained vectors ครับ

In [ ]:
'''ทำนาย test'''
pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)

In [ ]:
'''เคลียร์หน่วยความจำ'''
del embedding_matrix, model, x
import gc; gc.collect()
time.sleep(10)

**Wiki News FastText Embeddings:**

ต่อไปเราทดลอง FastText ครับ ซึ่งขั้นตอนก็จะเหมือนเดิมทุกประการ ดังนั้นเราจะไม่ขออธิบายอะไรเยอะครับ

In [ ]:
embedding_matrix =  load_fasttext_fast(tokenizer.word_index)
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=3, validation_data=(val_X, val_y))

In [ ]:
pred_fasttext_val_y = model.predict([val_X], batch_size=1024, verbose=1)

max_f1 = 0
for thresh in np.arange(0.2, 0.601, 0.01):
    thresh = np.round(thresh, 2)
    current_f1 = metrics.f1_score(val_y, (pred_fasttext_val_y>thresh).astype(int))
    print("ค่า F1 ที่ threshold %.2f is %.4f" % (thresh, current_f1) )
    
    if current_f1 > max_f1:
        best_thresh = thresh
        max_f1 = current_f1

print('ค่า threshold ที่ดีที่สุดคือ %.2f ให้ค่า F1 เท่ากับ %.4f' % (best_thresh,max_f1))


In [ ]:
pred_fasttext_test_y = model.predict([test_X], batch_size=1024, verbose=1)

In [ ]:
'''เคลียร์หน่วยความจำ'''
del embedding_matrix, model, x
import gc; gc.collect()
time.sleep(10)

**Paragram Embeddings:**

In this section, we can use the paragram embeddings and build the model and make predictions.

In [ ]:
embedding_matrix =  load_para_fast(tokenizer.word_index)
        
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(train_X, train_y, batch_size=512, epochs=3, validation_data=(val_X, val_y))

In [ ]:
pred_paragram_val_y = model.predict([val_X], batch_size=1024, verbose=1)

In [ ]:
max_f1 = 0
for thresh in np.arange(0.2, 0.601, 0.01):
    thresh = np.round(thresh, 2)
    current_f1 = metrics.f1_score(val_y, (pred_paragram_val_y>thresh).astype(int))
    print("ค่า F1 ที่ threshold %.2f is %.4f" % (thresh, current_f1) )
    
    if current_f1 > max_f1:
        best_thresh = thresh
        max_f1 = current_f1

print('ค่า threshold ที่ดีที่สุดคือ %.2f ให้ค่า F1 เท่ากับ %.4f' % (best_thresh,max_f1))

In [ ]:
pred_paragram_test_y = model.predict([test_X], batch_size=1024, verbose=1)

In [ ]:
'''เคลียร์หน่วยความจำ'''
del embedding_matrix, model, x
import gc; gc.collect()
time.sleep(10)

**ข้อสังเกต:**
 * การใช้ pretrained word vectors จะให้ประสิทธิภาพการทำนายโดยรวมที่ดีกว่าการไม่ใช้ pretrained word vectors 
 * ใน pretrained word vectors แต่ละชุดนั้น ประสิทธิภาพที่ได้ไม่ต่างกันมากเท่าไรนัก
 
## Ensemble

เทคนิกที่สำคัญมากๆ ในการเพิ่มประสิทธิภาพของการทำนาย คือเทคนิกที่เรียกว่า **ensemble** หรือการนำผลลัพธ์การทำนายของหลายๆ โมเดลมาเฉลี่ย (หรือมาโหวตกันก็ได้ครับ) ไอเดียของ ensemble คร่าวๆ ก็คือแม้ว่าแต่ละโมเดลที่ใช้ pretrained vectors ต่างชุดกัน จะให้ประสิทธิภาพไม่ต่างกันมาก แต่ว่าโมเดลเหล่านั้นอาจจะเข้าใจข้อมูลในแง่มุมที่แตกต่างกันไป ดังนั้นการนำทุกโมเดลมาช่วยกันทำนาย จึงเป็นการสร้างโมเดลใหม่ที่เข้าใจข้อมูลสอนได้ดีที่สุด


In [ ]:
'''สร้าง ensemble อย่างง่ายที่นำค่าความน่าจะเป็นของทั้งสามโมเดลมาเฉลี่ยกัน เพื่อให้ได้ค่าความน่าจะเป็นสุดท้าย'''
pred_val_y = 0.33*pred_glove_val_y + 0.33*pred_fasttext_val_y + 0.34*pred_paragram_val_y 

max_f1 = 0
for thresh in np.arange(0.2, 0.601, 0.01):
    thresh = np.round(thresh, 2)
    current_f1 = metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))
    print("ค่า F1 ที่ threshold %.2f is %.4f" % (thresh, current_f1) )
    
    if current_f1 > max_f1:
        best_thresh = thresh
        max_f1 = current_f1

print('ค่า threshold ที่ดีที่สุดคือ %.2f ให้ค่า F1 เท่ากับ %.4f' % (best_thresh,max_f1))

ผลการทำนายที่ได้ ดีขึ้นกว่าเดิมจริงๆ ด้วยครับ ดังนั้นเราจะทำนาย test data ด้วย ensemble model นี้กัน

In [ ]:
pred_test_y = 0.33*pred_glove_test_y + 0.33*pred_fasttext_test_y + 0.34*pred_paragram_test_y
pred_test_y = (pred_test_y>0.35).astype(int)
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)

จบแล้วครับ เมื่อรันเสร็จแล้ว เพื่อนๆ สามารถตรวจสอบประสิทธิภาพการทำนายบน test data ได้จากการ submit file submission.csv ตามรูปนี้ครับ โดยกดปุ่ม "Submit to competition" ได้เลยครับ
![กดปุ่ม submission ได้เลย](https://i.ibb.co/B39fYGx/1502078496147.jpg)

เท่าที่ผมรัน script นี้มาหลายรอบคะแนน public F1 score จะแกว่งอยู่ราวๆ 66%-68% (แกว่งมากเนื่ืองจากข้อมูลทดสอบใน public test นั้นมีน้อย) ส่วน private F1 score จะค่อนข้างนิ่งที่ 68%++ โดยคนที่ทำได้คะแนนระดับท้อปของโลกจะทำได้ราวๆ 70%-71%++ ซึ่งก็ถือว่าไม่ต่างกันมากในทางปฏิบัติสักเท่าไร อย่างไรก็ตามเพื่อนๆ อาจจะลองพัฒนาให้แม่นยำและครอบคลุมขึ้นได้ครับ

ถ้าใครอยากลองวิชาในการแข่งขันจริง Kaggle เพิ่งประกาศการแข่งขัน NLP classification ครั้งใหม่ สดๆ ร้อนๆ ที่นี่ครับ ซึ่งเพื่อนๆ สามารถลองประยุกต์ใช้ความรู้พื้นฐานจาก workshop นี้ในการแข่งจริง ข้อมูลจริงได้ทันทีครับ

https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification

การแข่งขัน Jigsaw toxicity classification อันใหม่นี้จะจบลงวันที่ 27 มิถุนายน 2562 ครับ

## Feedback
สุดท้ายนี้ เพื่อนๆ อยากให้ทีมงานทำ notebook workshop เกี่ยวกับปัญหา AI / Machine Learning ในด้านไหน เช่น Image Classification หรือ Signal Analysis หรืออยากให้ปรับปรุงอะไรสามารถแนะนำกันได้เต็มที่เลยนะครับ ไม่ว่าจะเป็นทาง thaikeras.com/community หรือ comments ด้านล่าง notebook ฉบับนี้ก็ได้ครับ